In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import lightgbm as lgb
import gc

In [14]:
#FILE
model_folder='eda/models/sample_run/'
model_file='specific_hours'
model_save=model_folder+model_file
metrics_file='eda/metrics.csv'

In [2]:
metrics = 'auc'

In [3]:
train_df=pd.read_csv('eda/features/train/all_features.csv',parse_dates=['click_date'])
val_df=pd.read_csv('eda/features/train/all_features.csv',parse_dates=['click_date'])
train_df=train_df.drop(['ip','click_time', 'attributed_time','click_date'],axis=1)
val_df=val_df.drop(['ip','click_time', 'attributed_time','click_date'],axis=1)

print(train_df.info())
print(val_df.info())

print("train size: ", len(train_df))
print("valid size: ", len(val_df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659779 entries, 0 to 659778
Data columns (total 14 columns):
app               659779 non-null int64
device            659779 non-null int64
os                659779 non-null int64
channel           659779 non-null int64
is_attributed     659779 non-null int64
hour              659779 non-null int64
day               659779 non-null int64
IN_TEST_HH        659779 non-null int64
IP_DAY_TEST_HH    659779 non-null int64
IP_DAY_HH         659779 non-null int64
IP_OS_HH          659779 non-null int64
IP_APP_HH         659779 non-null int64
IP_APP_OS_HH      659779 non-null int64
APP_DAY_HH        659779 non-null int64
dtypes: int64(14)
memory usage: 70.5 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659779 entries, 0 to 659778
Data columns (total 14 columns):
app               659779 non-null int64
device            659779 non-null int64
os                659779 non-null int64
channel           659779 non-null int64
is_attributed

In [4]:
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':metrics,
        'learning_rate': 0.1,
        'num_leaves': 7,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 4,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 100,  # Number of bucketed bin for feature values
        'subsample': 0.7,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'nthread': 4,
        'verbose': 0,
        'scale_pos_weight':99.7, # because training data is extremely unbalanced 
        'metric':metrics
}
target = 'is_attributed'
predictors = ['app', 'device', 'os', 'channel', 'hour',
       'day', 'IN_TEST_HH', 'IP_DAY_TEST_HH', 'IP_DAY_HH', 'IP_OS_HH',
       'IP_APP_HH', 'IP_APP_OS_HH', 'APP_DAY_HH']
categorical = ['app', 'device', 'os', 'channel', 'hour','day']


In [5]:
VALIDATE = True
MAX_ROUNDS = 500
EARLY_STOP = 50
OPT_ROUNDS = 500
print("Training...")

num_boost_round=MAX_ROUNDS
early_stopping_rounds=EARLY_STOP

xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del train_df
gc.collect()
print('done...')

Training...


201

done...


In [6]:
xgvalid = lgb.Dataset(val_df[predictors].values, label=val_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del val_df
gc.collect()

147

In [7]:
evals_results = {}
bst = lgb.train(lgb_params, 
                 xgtrain, 
                 valid_sets=[xgtrain, xgvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=num_boost_round,
                 early_stopping_rounds=early_stopping_rounds,
                 verbose_eval=10, 
                 feval=None)

n_estimators = bst.best_iteration
bst.save_model(model_save)
print("\nModel Report")
print("n_estimators : ", n_estimators)
print(metrics+":", evals_results['valid'][metrics][n_estimators-1])
del xgvalid

/Users/aronak/miniconda3/envs/ml_3_6/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/aronak/miniconda3/envs/ml_3_6/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[10]	train's auc: 0.958679	valid's auc: 0.958679
[20]	train's auc: 0.970926	valid's auc: 0.970926
[30]	train's auc: 0.976939	valid's auc: 0.976939
[40]	train's auc: 0.98202	valid's auc: 0.98202
[50]	train's auc: 0.984685	valid's auc: 0.984685
[60]	train's auc: 0.986009	valid's auc: 0.986009
[70]	train's auc: 0.987583	valid's auc: 0.987583
[80]	train's auc: 0.988926	valid's auc: 0.988926
[90]	train's auc: 0.990007	valid's auc: 0.990007
[100]	train's auc: 0.990857	valid's auc: 0.990857
[110]	train's auc: 0.991713	valid's auc: 0.991713
[120]	train's auc: 0.992293	valid's auc: 0.992293
[130]	train's auc: 0.992828	valid's auc: 0.992828
[140]	train's auc: 0.993422	valid's auc: 0.993422
[150]	train's auc: 0.993975	valid's auc: 0.993975
[160]	train's auc: 0.994316	valid's auc: 0.994316
[170]	train's auc: 0.994616	valid's auc: 0.994616
[180]	train's auc: 0.99487	valid's auc: 0.99487
[190]	train's auc: 0.99509	valid's auc: 0.99509
[20

NameError: name 'c_bst_file' is not defined

In [11]:
import datetime
date_time=datetime.datetime.now()
train_acc=evals_results['train']['auc'][n_estimators-1]
valid_acc=evals_results['valid']['auc'][n_estimators-1]
bst_iter=n_estimators
comment='1M, andys features,specific hours'

In [ ]:
#prepare metrics and see
col_list=['date_time','train_acc','valid_acc','best_iter','cols','model_file','comment']
curr_data=pd.DataFrame([[date_time,train_acc,valid_acc,bst_iter,predictors,model_save,comment]], columns=col_list)
curr_data.head()

In [ ]:
# ADD to records
add_to_records=False
log=pd.read_csv(metrics_file)
log.head()
if(add_to_records):
    log.append(curr_data)
    print('records updated')
    log.head()
    log.to_csv(metrics_file,index=False)
gc.collect()
#resetCSV
# curr_data.to_csv(metrics_file,index=False)
# curr_data.head()